In [1]:
import pandas as pd
import numpy as np

### Clean up candidate results

In [2]:
wmp = pd.read_csv("/Users/jieluyao/Documents/GitHub/datasets/candidates/2022/wmpcand_012523_wmpid.csv")

In [3]:
wmp.columns

Index(['wmpid', 'genelect_cd', 'CurrCand', 'cand_name', 'cand_id',
       'cand_office', 'cand_office_st', 'cand_office_dist',
       'cand_party_affiliation', 'cand_incumbent_challenger_open_s',
       'dateadded_cd', 'office_wapo', 'result_wapo', 'primarydate_wapo',
       'trumpe_wapo', 'gender_wmp', 'gender_crp', 'latino_wmp', 'latino_crp',
       'race_wmp', 'race_crp1', 'race_crp2', 'race_crpmena', 'hse_cmpt_gen',
       'full_name', 'first_name', 'last_name'],
      dtype='object')

In [4]:
wmp.loc[wmp['wmpid']=='WMPID2092']

,wmpid,genelect_cd,CurrCand,cand_name,cand_id,cand_office,cand_office_st,cand_office_dist,cand_party_affiliation,cand_incumbent_challenger_open_s,...,latino_wmp,latino_crp,race_wmp,race_crp1,race_crp2,race_crpmena,hse_cmpt_gen,full_name,first_name,last_name
2557,WMPID2092,1,Y,"SLUZYNSKY, WALTER STEVE MR.",H2PA17194,H,PA,17.0,UP,CHALLENGER,...,NaN,NaN,NaN,NaN,NaN,NaN,1,Walter Steve Sluzynsky,Walter Steve,Sluzynsky
4133,WMPID2092,1,Y,"SLUZYNSKY, WALTER STEVE MR.",S2PA00612,S,PA,17.0,DEM,OPEN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,Walter Steve Sluzynsky,Walter Steve,Sluzynsky


In [5]:
territories = ['AS', 'DC', 'GU', 'MP', 'PR', 'VI']

In [8]:
wmp = wmp.loc[~wmp['cand_office_st'].isin(territories)]

In [9]:
wmp.shape

(4277, 27)

In [51]:
wmp.loc[(wmp['genelect_cd']==1)].shape

(1356, 27)

### Senate

In [10]:
s = wmp.loc[(wmp['genelect_cd']==1)&(wmp['cand_office']=='S')]

In [11]:
s.shape

(173, 27)

Clean up Jasmine's work

In [12]:
hc_s = pd.read_excel("bp2022_sen_scraped_face_jasmine.xlsx")

In [13]:
hc_s['bp_url_final'] = np.where(hc_s['bp_url'].isna(), hc_s['hc_face_url'], hc_s['bp_url'])

In [14]:
hc_s.loc[hc_s['bp_url_final'].isna()].shape

(12, 21)

In [15]:
# Replace place holder image with NAN
hc_s.loc[hc_s['bp_url_final'].str.contains('Placeholder', na=False), 'bp_url_final']= np.NAN

In [16]:
hc_s.loc[hc_s['bp_url_final'].isna()].shape

(33, 21)

In [17]:
hc_s.shape

(172, 21)

In [18]:
hc_s = hc_s[['wmpid','full_name', 'bp_url_final']]

Merge results

In [19]:
senate = s.merge(hc_s, on='wmpid', how='left', indicator=True)

In [20]:
senate._merge.value_counts()

both          172
left_only       1
right_only      0
Name: _merge, dtype: int64

In [21]:
senate.loc[senate['_merge']=='left_only']

,wmpid,genelect_cd,CurrCand,cand_name,cand_id,cand_office,cand_office_st,cand_office_dist,cand_party_affiliation,cand_incumbent_challenger_open_s,...,race_crp1,race_crp2,race_crpmena,hse_cmpt_gen,full_name_x,first_name,last_name,full_name_y,bp_url_final,_merge
125,WMPID2092,1,Y,"SLUZYNSKY, WALTER STEVE MR.",S2PA00612,S,PA,17.0,DEM,OPEN,...,NaN,NaN,NaN,1,Walter Steve Sluzynsky,Walter Steve,Sluzynsky,NaN,NaN,left_only


In [22]:
senate.loc[senate['full_name_x']!=senate['full_name_y']]

,wmpid,genelect_cd,CurrCand,cand_name,cand_id,cand_office,cand_office_st,cand_office_dist,cand_party_affiliation,cand_incumbent_challenger_open_s,...,race_crp1,race_crp2,race_crpmena,hse_cmpt_gen,full_name_x,first_name,last_name,full_name_y,bp_url_final,_merge
125,WMPID2092,1,Y,"SLUZYNSKY, WALTER STEVE MR.",S2PA00612,S,PA,17.0,DEM,OPEN,...,NaN,NaN,NaN,1,Walter Steve Sluzynsky,Walter Steve,Sluzynsky,NaN,NaN,left_only


In [23]:
senate.columns

Index(['wmpid', 'genelect_cd', 'CurrCand', 'cand_name', 'cand_id',
       'cand_office', 'cand_office_st', 'cand_office_dist',
       'cand_party_affiliation', 'cand_incumbent_challenger_open_s',
       'dateadded_cd', 'office_wapo', 'result_wapo', 'primarydate_wapo',
       'trumpe_wapo', 'gender_wmp', 'gender_crp', 'latino_wmp', 'latino_crp',
       'race_wmp', 'race_crp1', 'race_crp2', 'race_crpmena', 'hse_cmpt_gen',
       'full_name_x', 'first_name', 'last_name', 'full_name_y', 'bp_url_final',
       '_merge'],
      dtype='object')

In [24]:
senate =senate[['wmpid', 'full_name_x', '_merge', 'bp_url_final']]

### House

In [26]:
h = wmp.loc[(wmp['genelect_cd']==1)&(wmp['cand_office']=='H')]

In [27]:
h.shape

(1183, 27)

Clean up Jasmine's work

In [28]:
hc_h = pd.read_excel("bp2022_house_scraped_face_jasmine.xlsx")

In [29]:
hc_h['bp_url_final'] = np.where(hc_h['bp_url'].isna(), hc_h['hc_face_url'], hc_h['bp_url'])

In [30]:
hc_h.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,...,hc_first_name,hc_last_name,hc_middle_name,hc_suffix,hc_affix,hc_name_other,hc_double_surname,hc_office_district,hc_office_district_note,bp_url_final
0,WMPID2014,"PALIN, SARAH",Sarah Palin,AK,0,REP,Sarah Palin,Sarah Palin,https://s3.amazonaws.com/ballotpedia-api4/file...,Sarah Palin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://s3.amazonaws.com/ballotpedia-api4/file...
1,WMPID4003,"BEGICH, NICHOLAS III",Nicholas III Begich,AK,0,REP,Nicholas Begich,Nicholas Begich,https://s3.amazonaws.com/ballotpedia-api4/file...,Nicholas Begich,...,Nicholas,Begich,NaN,III,NaN,NaN,NaN,NaN,NaN,https://s3.amazonaws.com/ballotpedia-api4/file...
2,WMPID4308,"PELTOLA, MARY",Mary Peltola,AK,0,DEM,Mary Peltola,Mary Peltola,https://s3.amazonaws.com/ballotpedia-api4/file...,Mary Peltola,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://s3.amazonaws.com/ballotpedia-api4/file...
3,WMPID4360,"SWEENEY, TARA M",Tara Sweeney,AK,0,REP,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://s3.amazonaws.com/ballotpedia-api4/file...
4,WMPID5148,"ROGERS, DAVID L.",David Rogers,DE,0,OTH,David Rogers,David Rogers (Nonpartisan Party) \n\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,David Rogers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://s3.amazonaws.com/ballotpedia-api4/file...


In [31]:
hc_h.loc[hc_h['bp_url_final'].isna()].shape

(13, 24)

In [32]:
# Replace place holder image with NAN
hc_h.loc[hc_h['bp_url_final'].str.contains('Placeholder', na=False), 'bp_url_final']= np.NAN

In [33]:
hc_h.loc[hc_h['bp_url_final'].isna()].shape

(58, 24)

In [34]:
hc_h.loc[hc_h['bp_url_final'].isna()]

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,...,hc_first_name,hc_last_name,hc_middle_name,hc_suffix,hc_affix,hc_name_other,hc_double_surname,hc_office_district,hc_office_district_note,bp_url_final
14,WMPID5069,"NOLFI, BARBARA",Barbara Nolfi,VT,0,PRO,NaN,NaN,https://cdn.ballotpedia.org/images/f/fb/Silhou...,Barbara Nolfi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,WMPID4534,"SMITH, JONATHAN",Jonathan Smith,AL,2,LIB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,WMPID4177,"FOSTER, WILLIAM",William Foster,AZ,6,IND,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,WMPID2895,"DAUGHERTY, ROGER B",Roger Daugherty,AR,1,IND,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,WMPID4632,"THOMPSON, RYAN CASEY",Ryan Casey Thompson,CO,5,UN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,WMPID1946,"WHITE, ANDREW FORREST",Andrew Forrest White,CO,6,IND,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
398,WMPID3733,"LITCHFIELD, GEORGE E MR. II",George II Litchfield,GA,1,CRV,NaN,NaN,NaN,NaN,...,George,Litchfield,NaN,II,NaN,NaN,NaN,NaN,NaN,NaN
401,WMPID3830,"PACETTI, ANDREW MR",Andrew Pacetti,GA,3,UN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,WMPID1702,"MOSLEY, MARK",Mark Mosley,GA,8,LIB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
406,WMPID4328,"COPE, MICHELLE",Michelle Cope,GA,8,LIB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
hc_h.shape

(1286, 24)

In [36]:
hc_h = hc_h[['wmpid','full_name', 'bp_url_final']]

Merge results

In [37]:
house = h.merge(hc_h, on='wmpid', how='left', indicator=True)

In [38]:
house._merge.value_counts()

both          1181
left_only        2
right_only       0
Name: _merge, dtype: int64

In [39]:
house.loc[house['full_name_x']!=house['full_name_y']]

,wmpid,genelect_cd,CurrCand,cand_name,cand_id,cand_office,cand_office_st,cand_office_dist,cand_party_affiliation,cand_incumbent_challenger_open_s,...,race_crp1,race_crp2,race_crpmena,hse_cmpt_gen,full_name_x,first_name,last_name,full_name_y,bp_url_final,_merge
184,WMPID4631,1,Y,"BYE, CHRIS",H2AK01216,H,AK,0.0,LIB,OPEN,...,NaN,NaN,NaN,0,Chris Bye,Chris,Bye,NaN,NaN,left_only
289,WMPID2605,1,Y,"LONG, TIMOTHY LOUIS",H2CO08145,H,CO,8.0,UN,OPEN,...,NaN,NaN,NaN,1,Timothy Louis Long,Timothy Louis,Long,NaN,NaN,left_only


In [40]:
house.loc[house['_merge']=='left_only']

,wmpid,genelect_cd,CurrCand,cand_name,cand_id,cand_office,cand_office_st,cand_office_dist,cand_party_affiliation,cand_incumbent_challenger_open_s,...,race_crp1,race_crp2,race_crpmena,hse_cmpt_gen,full_name_x,first_name,last_name,full_name_y,bp_url_final,_merge
184,WMPID4631,1,Y,"BYE, CHRIS",H2AK01216,H,AK,0.0,LIB,OPEN,...,NaN,NaN,NaN,0,Chris Bye,Chris,Bye,NaN,NaN,left_only
289,WMPID2605,1,Y,"LONG, TIMOTHY LOUIS",H2CO08145,H,CO,8.0,UN,OPEN,...,NaN,NaN,NaN,1,Timothy Louis Long,Timothy Louis,Long,NaN,NaN,left_only


In [41]:
house =house[['wmpid', 'full_name_x', '_merge', 'bp_url_final']]

In [57]:
c = pd.concat([senate, house])

In [58]:
c.head()

,wmpid,full_name_x,_merge,bp_url_final
0,WMPID1299,John Boozman,both,https://s3.amazonaws.com/ballotpedia-api4/file...
1,WMPID1302,Mark Kelly,both,https://s3.amazonaws.com/ballotpedia-api4/file...
2,WMPID1305,Michael Bennet,both,https://s3.amazonaws.com/ballotpedia-api4/file...
3,WMPID1315,Richard Blumenthal,both,https://s3.amazonaws.com/ballotpedia-api4/file...
4,WMPID1321,Marco Rubio,both,https://s3.amazonaws.com/ballotpedia-api4/file...


In [59]:
len(c.wmpid.unique())

1355

In [60]:
c.shape

(1356, 4)

In [61]:
c.loc[c['wmpid'].duplicated(keep=False)]
# Walter Steve Sluzynsky ran for both House and Senate

,wmpid,full_name_x,_merge,bp_url_final
125,WMPID2092,Walter Steve Sluzynsky,left_only,NaN
751,WMPID2092,Walter Steve Sluzynsky,both,https://s3.amazonaws.com/ballotpedia-api4/file...


In [62]:
c.loc[c['wmpid'].duplicated() == True, 'bp_url_final'][751]

'https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/100/100/Christopher-DeLuzio.PNG'

In [63]:
c = c.drop([751])

In [64]:
c.loc[c['wmpid'].duplicated(keep=False)]

,wmpid,full_name_x,_merge,bp_url_final


In [65]:
c.shape

(1355, 4)

In [66]:
c.loc[c['_merge']=='left_only']

,wmpid,full_name_x,_merge,bp_url_final
125,WMPID2092,Walter Steve Sluzynsky,left_only,NaN
184,WMPID4631,Chris Bye,left_only,NaN
289,WMPID2605,Timothy Louis Long,left_only,NaN


Add missing urls

In [67]:
c.loc[c['wmpid']=='WMPID4631', 'bp_url_final'] = 'https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/Chris-Bye.PNG'

In [68]:
c.loc[c['wmpid']=='WMPID2605', 'bp_url_final'] ='https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/200/300/TimLong.jpg'

In [69]:
c.loc[c['_merge']=='left_only']

,wmpid,full_name_x,_merge,bp_url_final
125,WMPID2092,Walter Steve Sluzynsky,left_only,NaN
184,WMPID4631,Chris Bye,left_only,https://s3.amazonaws.com/ballotpedia-api4/file...
289,WMPID2605,Timothy Louis Long,left_only,https://s3.amazonaws.com/ballotpedia-api4/file...


### Clean up people results

In [70]:
pple = pd.read_csv("/Users/jieluyao/Documents/GitHub/datasets/candidates/2022/wmpcand_012523_wmpid.csv")

In [71]:
pple.head()

,wmpid,genelect_cd,CurrCand,cand_name,cand_id,cand_office,cand_office_st,cand_office_dist,cand_party_affiliation,cand_incumbent_challenger_open_s,...,latino_wmp,latino_crp,race_wmp,race_crp1,race_crp2,race_crpmena,hse_cmpt_gen,full_name,first_name,last_name
0,WMPID21,1,Y,"CARL, JERRY LEE, JR",H0AL01055,H,AL,1.0,REP,INCUMBENT,...,0.0,N,White,W,NaN,N,0,Jerry Carl,Jerry,Carl
1,WMPID24,1,Y,"HARVEY-HALL, PHYLLIS",H0AL02202,H,AL,2.0,DEM,CHALLENGER,...,0.0,n,Black,B,NaN,n,0,Phyllis Harvey-Hall,Phyllis,Harvey-Hall
2,WMPID27,1,Y,"SEWELL, TERRI A.",H0AL07086,H,AL,7.0,DEM,INCUMBENT,...,0.0,N,Black,B,NaN,N,0,Terri A Sewell,Terri,Sewell
3,WMPID28,1,Y,"CRAWFORD, ERIC ALAN RICK",H0AR01083,H,AR,1.0,REP,INCUMBENT,...,0.0,N,White,W,NaN,N,0,Rick Crawford,Rick,Crawford
4,WMPID30,1,Y,"WOMACK, STEVE",H0AR03055,H,AR,3.0,REP,INCUMBENT,...,0.0,N,White,W,NaN,N,0,Steve Womack,Steve,Womack


In [72]:
pple.columns

Index(['wmpid', 'genelect_cd', 'CurrCand', 'cand_name', 'cand_id',
       'cand_office', 'cand_office_st', 'cand_office_dist',
       'cand_party_affiliation', 'cand_incumbent_challenger_open_s',
       'dateadded_cd', 'office_wapo', 'result_wapo', 'primarydate_wapo',
       'trumpe_wapo', 'gender_wmp', 'gender_crp', 'latino_wmp', 'latino_crp',
       'race_wmp', 'race_crp1', 'race_crp2', 'race_crpmena', 'hse_cmpt_gen',
       'full_name', 'first_name', 'last_name'],
      dtype='object')

In [75]:
pple2 = pd.read_csv('../person_2022_cd010623.csv')

In [76]:
pple2.head()

,wmpid,full_name,name_fec,first_name,last_name,gender_wmp,fecid_2020,fecid_2022a,fecid_2022b,fecid_2022old,...,former_uspres,intl_leaders,gov2022_gencd,face_category,latino_wmp,latino_crp,race_wmp,race_crp1,race_crp2,race_crpmena
0,WMPID4735,A.J. Calkins,"CALKINS, AARON (A.J.)",A.J.,Calkins,M,NaN,H2IN03236,NaN,NaN,...,0.0,0.0,0,NaN,NaN,NaN,White,W,NaN,NaN
1,WMPID1731,Aamir Sultan,"SULTAN, AAMIR",Aamir,Sultan,UNK,NaN,H2NY10191,NaN,NaN,...,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WMPID4403,Aarika Samone Rhodes,"RHODES, AARIKA SAMONE",Aarika Samone,Rhodes,F,NaN,H2CA30275,NaN,NaN,...,0.0,0.0,0,NaN,0.0,N,Black,B,NaN,N
3,WMPID328,Aaron Amede,"AMEDE, AARON",Aaron,Amede,UNK,H0ME01127,H0ME01127,NaN,NaN,...,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WMPID1812,Aaron Bean,"BEAN, AARON P.",Aaron,Bean,M,NaN,H2FL04211,NaN,NaN,...,0.0,0.0,0,NaN,0.0,N,White,W,NaN,N


In [77]:
pple2.columns

Index(['wmpid', 'full_name', 'name_fec', 'first_name', 'last_name',
       'gender_wmp', 'fecid_2020', 'fecid_2022a', 'fecid_2022b',
       'fecid_2022old', 'fecid_changed', 'crp_candidate_id',
       'dateadded_person', 'notes_person', 'wmp_notes', 'genelect_cd_2022',
       'supcourt_2022', 'supcourt_former', 'currsen_2022', 'prompol',
       'former_uspres', 'intl_leaders', 'gov2022_gencd', 'face_category',
       'latino_wmp', 'latino_crp', 'race_wmp', 'race_crp1', 'race_crp2',
       'race_crpmena'],
      dtype='object')

In [78]:
pple2.gov2022_gencd.value_counts()

0    5253
1      82
Name: gov2022_gencd, dtype: int64

In [138]:
pple2.loc[(pple2['supcourt_2022']==1) | (pple2['supcourt_former']==1)].shape

(16, 30)

### Clean up gubnatorial results

In [120]:
gov = pple2.loc[pple2['gov2022_gencd']==1]

In [121]:
gov.shape

(82, 30)

In [122]:
gov.head()

,wmpid,full_name,name_fec,first_name,last_name,gender_wmp,fecid_2020,fecid_2022a,fecid_2022b,fecid_2022old,...,former_uspres,intl_leaders,gov2022_gencd,face_category,latino_wmp,latino_crp,race_wmp,race_crp1,race_crp2,race_crpmena
387,WMPID5267,Ashley Kalus,"KALUS, ASHLEY",Ashley,Kalus,F,NaN,NaN,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
479,WMPID5285,Bernard Peters,"PETERS, BERNARD",Bernard,Peters,M,NaN,NaN,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
492,WMPID5331,Beto O'Rourke,"O'ROURKE, BETO",Beto,O'Rourke,M,NaN,NaN,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
495,WMPID5338,Betsy Johnson,"JOHNSON, BETSY",Betsy,Johnson,F,NaN,NaN,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
517,WMPID5280,Bill Lee,"LEE, BILL",Bill,Lee,M,NaN,NaN,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
len(gov['wmpid'].unique())

82

In [125]:
gov.loc[gov['wmpid'].isna()]

,wmpid,full_name,name_fec,first_name,last_name,gender_wmp,fecid_2020,fecid_2022a,fecid_2022b,fecid_2022old,...,former_uspres,intl_leaders,gov2022_gencd,face_category,latino_wmp,latino_crp,race_wmp,race_crp1,race_crp2,race_crpmena


In [126]:
g = pd.read_csv('bp2022_gov_scraped_face_verified.csv')

In [127]:
g.shape

(82, 7)

In [128]:
g.columns

Index(['abc_name', 'abc_name_raw', 'abc_state', 'bp_name', 'bp_name_raw',
       'bp_url', 'bp_url_filename'],
      dtype='object')

In [131]:
governor = gov.merge(g, left_on='full_name', right_on='bp_name', how='left', indicator=True)

In [132]:
governor._merge.value_counts()

both          82
left_only      0
right_only     0
Name: _merge, dtype: int64

In [133]:
governor.columns

Index(['wmpid', 'full_name', 'name_fec', 'first_name', 'last_name',
       'gender_wmp', 'fecid_2020', 'fecid_2022a', 'fecid_2022b',
       'fecid_2022old', 'fecid_changed', 'crp_candidate_id',
       'dateadded_person', 'notes_person', 'wmp_notes', 'genelect_cd_2022',
       'supcourt_2022', 'supcourt_former', 'currsen_2022', 'prompol',
       'former_uspres', 'intl_leaders', 'gov2022_gencd', 'face_category',
       'latino_wmp', 'latino_crp', 'race_wmp', 'race_crp1', 'race_crp2',
       'race_crpmena', 'abc_name', 'abc_name_raw', 'abc_state', 'bp_name',
       'bp_name_raw', 'bp_url', 'bp_url_filename', '_merge'],
      dtype='object')

In [134]:
len(governor['wmpid'].unique())

82

In [96]:
governor = governor[['wmpid', 'full_name', 'bp_url', '_merge']]

In [97]:
governor.head()

,wmpid,full_name,bp_url,_merge
0,WMPID5267,Ashley Kalus,https://s3.amazonaws.com/ballotpedia-api4/file...,both
1,WMPID5331,Beto O'Rourke,https://s3.amazonaws.com/ballotpedia-api4/file...,both
2,WMPID5280,Bill Lee,https://s3.amazonaws.com/ballotpedia-api4/file...,both
3,WMPID5288,Bob Stefanowski,https://s3.amazonaws.com/ballotpedia-api4/file...,both
4,WMPID5290,Brad Little,https://s3.amazonaws.com/ballotpedia-api4/file...,both


In [119]:
governor.shape

(66, 4)

### Clean up other politicians

In [100]:
pwd

'/Users/jieluyao/Documents/GitHub/face_url_scraper_2022/ballotpedia'

In [101]:
o = pd.read_csv("../non_cand_face_urls.csv")

In [102]:
o.head()

,face_name,face_url,face_source,face_category,wmpid_status
0,Alberto Fernández,https://upload.wikimedia.org/wikipedia/commons...,https://en.wikipedia.org/wiki/G20,international leaders,No
1,Anthony Albanese,https://upload.wikimedia.org/wikipedia/commons...,https://en.wikipedia.org/wiki/G20,international leaders,No
2,Jair Bolsonaro,https://upload.wikimedia.org/wikipedia/commons...,https://en.wikipedia.org/wiki/G20,international leaders,No
3,Justin Trudeau,https://upload.wikimedia.org/wikipedia/commons...,https://en.wikipedia.org/wiki/G20,international leaders,No
4,Xi Jinping,https://upload.wikimedia.org/wikipedia/commons...,https://en.wikipedia.org/wiki/G20,international leaders,No


In [103]:
other = pple2.merge(o, left_on='full_name', right_on='face_name', indicator=True)

In [104]:
other._merge.value_counts()

both          105
left_only       0
right_only      0
Name: _merge, dtype: int64

In [105]:
other = other.loc[other['_merge']=='both']

In [106]:
other.shape

(105, 36)

In [107]:
other.head()

,wmpid,full_name,name_fec,first_name,last_name,gender_wmp,fecid_2020,fecid_2022a,fecid_2022b,fecid_2022old,...,race_wmp,race_crp1,race_crp2,race_crpmena,face_name,face_url,face_source,face_category_y,wmpid_status,_merge
0,WMPID5239,Abraham Lincoln,"LINCOLN, ABRAHAM",Abraham,Lincoln,M,NaN,NaN,NaN,NaN,...,White,NaN,NaN,NaN,Abraham Lincoln,https://www.whitehouse.gov/wp-content/uploads/...,https://www.whitehouse.gov/about-the-white-hou...,former presidents,No,both
1,WMPID5195,Alberto Fernández,"FERNÁNDEZ, ALBERTO",Alberto,Fernández,M,NaN,NaN,NaN,NaN,...,Hispanic/Latino,NaN,NaN,NaN,Alberto Fernández,https://upload.wikimedia.org/wikipedia/commons...,https://en.wikipedia.org/wiki/G20,international leaders,No,both
2,WMPID5203,Alejandro Mayorkas,"MAYORKAS, ALEJANDRO",Alejandro,Mayorkas,M,NaN,NaN,NaN,NaN,...,Hispanic/Latino,NaN,NaN,NaN,Alejandro Mayorkas,https://www.whitehouse.gov/wp-content/uploads/...,https://www.whitehouse.gov/administration/cabi...,cabinet,No,both
3,WMPID5204,Andrew Jackson,"JACKSON, ANDREW",Andrew,Jackson,M,NaN,NaN,NaN,NaN,...,White,NaN,NaN,NaN,Andrew Jackson,https://www.whitehouse.gov/wp-content/uploads/...,https://www.whitehouse.gov/about-the-white-hou...,former presidents,No,both
4,WMPID5213,Andrew Johnson,"JOHNSON, ANDREW",Andrew,Johnson,M,NaN,NaN,NaN,NaN,...,White,NaN,NaN,NaN,Andrew Johnson,https://www.whitehouse.gov/wp-content/uploads/...,https://www.whitehouse.gov/about-the-white-hou...,former presidents,No,both


In [108]:
other = other[['wmpid', 'face_url', 'face_category_y']]

In [109]:
other.head()

,wmpid,face_url,face_category_y
0,WMPID5239,https://www.whitehouse.gov/wp-content/uploads/...,former presidents
1,WMPID5195,https://upload.wikimedia.org/wikipedia/commons...,international leaders
2,WMPID5203,https://www.whitehouse.gov/wp-content/uploads/...,cabinet
3,WMPID5204,https://www.whitehouse.gov/wp-content/uploads/...,former presidents
4,WMPID5213,https://www.whitehouse.gov/wp-content/uploads/...,former presidents


In [110]:
other.shape

(105, 3)

In [111]:
other['face_category_y'].value_counts()

former presidents         44
international leaders     23
cabinet                   22
supreme court justices     9
public health related      6
historical figures         1
Name: face_category_y, dtype: int64

In [116]:
other.loc[other['face_category_y']=='former presidents', 'face_url'][83]

'https://www.whitehouse.gov/wp-content/uploads/2021/01/40_ronald_reagan.jpg'

In [118]:
other.loc[other['face_category_y']=='public health related', 'face_url'][7]

'https://upload.wikimedia.org/wikipedia/commons/thumb/6/61/Anthony_Fauci_2020.jpg/220px-Anthony_Fauci_2020.jpg'

### Current senators

In [114]:
pple2.loc[pple2['currsen_2022']==1]

,wmpid,full_name,name_fec,first_name,last_name,gender_wmp,fecid_2020,fecid_2022a,fecid_2022b,fecid_2022old,...,former_uspres,intl_leaders,gov2022_gencd,face_category,latino_wmp,latino_crp,race_wmp,race_crp1,race_crp2,race_crpmena
132,WMPID1456,Alex Padilla,"PADILLA, ALEX",Alex,Padilla,M,S2CA00955,S2CA00955,NaN,NaN,...,0.0,0.0,0,NaN,1.0,Y,Hispanic/Latino,H,NaN,N
201,WMPID1512,Amy Klobuchar,"KLOBUCHAR, AMY",Amy,Klobuchar,F,S6MN00267,NaN,NaN,NaN,...,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,WMPID1461,Angus King,"KING, ANGUS",Angus,King,M,S2ME00109,NaN,NaN,NaN,...,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
454,WMPID1412,Ben Ray Lujan,"LUJAN, BEN RAY",Ben Ray,Lujan,M,S0NM00058,NaN,NaN,NaN,...,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455,WMPID1487,Ben Sasse,"SASSE, BEN",Ben,Sasse,M,S4NE00090,NaN,NaN,NaN,...,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5017,WMPID1495,Tim Scott,"SCOTT, TIMOTHY E.",Tim,Scott,M,S4SC00240,S4SC00240,NaN,NaN,...,0.0,0.0,0,NaN,NaN,NaN,Black,B,NaN,NaN
5047,WMPID1539,Tina Smith,"SMITH, TINA",Tina,Smith,F,S8MN00578,NaN,NaN,NaN,...,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5056,WMPID1505,Todd Young,"YOUNG, TODD CHRISTOPHER",Todd,Young,M,S6IN00191,S6IN00191,NaN,NaN,...,0.0,0.0,0,NaN,NaN,NaN,White,W,NaN,NaN
5060,WMPID1473,Tom Cotton,"COTTON, TOM",Tom,Cotton,M,S4AR00103,NaN,NaN,NaN,...,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
